# Import required packages

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold,cross_validate, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
#from yellowbrick.classifier import ClassificationReport
import warnings
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
from scipy import stats

# Data preprocessing stage - merging and cleaning of data

### Data Collection and Preprocessing

In [2]:
#merging all the required datasets - metabolomics dataset, drug response dataset and the metadata of the cell lines

data = pd.read_excel("D:\\GaTech\\Fall Sem\\BIOL 8901\\Metabolomic Project\\metabolomic_data.xlsx", sheet_name="1-clean data")
data.rename(columns = {'Unnamed: 0': 'ID'}, inplace = True)
last_column = data.iloc[:,-1].name
sample = pd.read_csv("D:\\GaTech\\Fall Sem\\BIOL 8901\\sample_info.csv")
sample.rename(columns = {'CCLE_Name':'ID'}, inplace=True)
merged_data = data.merge(sample, on='ID')
drug = pd.read_csv('D:\\GaTech\\Fall Sem\\BIOL 8901\\sanger-dose-response.csv')
drug.rename(columns={'ARXSPAN_ID':'DepMap_ID'}, inplace = True)
working_data = merged_data.merge(drug, on='DepMap_ID')

In [3]:
#since our data has a lot of NaN's in them, we can fill them using 0 (for now, just for a workaround)

working_data.fillna(0, inplace=True)

In [4]:
X1 = working_data.loc[working_data['DRUG_NAME'] == 'CISPLATIN']

In [5]:
'''
reset the index as once we take a subset of the main working_dataset, the indexes will get mixed up.
thus the indices need to be reset before we start working on the model
'''

X1.reset_index(inplace=True)

### Remove GDSC1 data (per GDSC instructions) and keep GDSC2 data only

In [6]:
#wherever there is any duplicate within a cell line's IC50 values, take the cell line that originates from the GDSC2 phase 
X_ = X1[~X1.duplicated(['DepMap_ID'], keep=False) | X1['DATASET'].eq('GDSC2')]

In [7]:
X_.reset_index(inplace=True)

In [8]:
#drop the metadata

X_ = X_.select_dtypes('float64')

In [9]:
X_.reset_index(inplace=True)

In [10]:
#scale the dataframe, at a quick glance the metabolite profiles are not scaled thus scaling of the dataframe is necessary

X_min = X_.min()
X_max = X_.max()
X_range = (X_max-X_min)
X_scaled = (X_-X_min)/(X_range)

In [11]:
#calculate the mean and standard deviation of the IC50_PUBLISHED

drug_mean = X_scaled['IC50_PUBLISHED'].mean()
drug_std = X_scaled['IC50_PUBLISHED'].std()
print(drug_mean, drug_std)

0.019814174656500174 0.07151777667799389


### Label Creation

In [12]:
#create labels

ic, labels = X_scaled['IC50_PUBLISHED'], []
for i in range(len(ic)):
    #if the IC50_PUBLISHED value is greater than the mean, add responsive label to the label list
    if ic[i] > drug_mean:
        labels.append('R')
    #if the IC50_PUBLISHED value is less than the mean, add non responsive label to the label list
    elif ic[i] < drug_mean:
        labels.append('NR')
        
y = pd.Series(labels)

In [13]:
y.value_counts()

NR    513
R     105
dtype: int64

In [14]:
X = X_.iloc[:, :225]
X.reset_index(inplace=True)

### Dimension reduction using Correlation

In [15]:
#calculate the correlation matrix of the metabolite dataframe
#choose the upper triangle of the correlation matrix
#create a list of features where the correlation value is >0.90
#this list contains the highly correlated features, which will be removed from the dataset

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column]>0.90)]

In [16]:
len(to_drop)

21

In [17]:
#drop the list of highly correlated features computed above

X.drop(to_drop, axis=1, inplace=True)

In [18]:
X.shape, y.shape

((618, 205), (618,))

## Data Processing

### Feature Selection using Recursive Feature Elimination using Cross Validation

In [19]:
#run the RFECV model with estimator being Random Forest and StratifiedKFold cross validation with 5 folds.

rfecv = RFECV(estimator = RandomForestClassifier(random_state=101), step=1, cv=StratifiedKFold(5), scoring='accuracy')

In [20]:
#fit the X,y to the RFECV model

rfecv.fit(X, y)

RFECV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
      estimator=RandomForestClassifier(random_state=101), scoring='accuracy')

In [21]:
#choose the features that are selected by the RFECV model

selected_features = rfecv.get_support(1)

In [22]:
#select a subset dataframe that contains only the "optimal" metabolic features returned from the RFECV model

X3 = X[X.columns[selected_features]]
X3.shape

(618, 62)

### Random Forest 

### Model training and Classification

In [23]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is Random Forest

kf3 = StratifiedKFold(n_splits = 5, shuffle=False)
model=RandomForestClassifier()
i=1
dfs = []
for train_index, test_index in kf3.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model.fit(X_train, y_train)
    #predict the test dataset
    predicted = model.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs.append(df)
    i+=1
results_df = pd.concat(dfs)

In [24]:
results_df

,precision,recall,f1-score,support
NR,0.834711,0.980583,0.901786,103.000000
R,0.333333,0.047619,0.083333,21.000000
accuracy,0.822581,0.822581,0.822581,0.822581
macro avg,0.584022,0.514101,0.492560,124.000000
weighted avg,0.749800,0.822581,0.763177,124.000000
NR,0.830645,1.000000,0.907489,103.000000
R,0.000000,0.000000,0.000000,21.000000
accuracy,0.830645,0.830645,0.830645,0.830645
macro avg,0.415323,0.500000,0.453744,124.000000
weighted avg,0.689971,0.830645,0.753801,124.000000


In [25]:
#results_df.to_csv("CISPLATIN_Results_RF.tsv", sep="\t")

### SVM

### Model training and classification

In [26]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is Support Vector Machine

kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model2=svm.SVC(kernel='linear', C=100)
i=1
dfs_svm = []
for train_index, test_index in kf5.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model2.fit(X_train, y_train)
    #predict the test dataset
    predicted2 = model2.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted2, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_svm.append(df)
    i+=1
results_df_svm = pd.concat(dfs_svm)

In [27]:
results_df_svm

,precision,recall,f1-score,support
NR,0.887500,0.689320,0.775956,103.000000
R,0.272727,0.571429,0.369231,21.000000
accuracy,0.669355,0.669355,0.669355,0.669355
macro avg,0.580114,0.630374,0.572594,124.000000
weighted avg,0.783385,0.669355,0.707075,124.000000
NR,0.873874,0.941748,0.906542,103.000000
R,0.538462,0.333333,0.411765,21.000000
accuracy,0.838710,0.838710,0.838710,0.838710
macro avg,0.706168,0.637540,0.659153,124.000000
weighted avg,0.817070,0.838710,0.822749,124.000000


### AdaBoost

### Model training and classification

In [28]:
#using classification_report metrics, run a prediction model using StratifiedKFold cross_validation with k=5 folds
#model being used as the classifier is AdaBoost Algorithm

kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model3=AdaBoostClassifier(n_estimators=500, learning_rate=0.8)
i=1
dfs_adb = []
for train_index, test_index in kf5.split(X3, y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    model3.fit(X_train, y_train)
    #predict the test dataset
    predicted3 = model3.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted3, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_adb.append(df)
    i+=1
results_df_adb = pd.concat(dfs_adb)

In [29]:
results_df_adb

,precision,recall,f1-score,support
NR,0.865979,0.815534,0.840000,103.000000
R,0.296296,0.380952,0.333333,21.000000
accuracy,0.741935,0.741935,0.741935,0.741935
macro avg,0.581138,0.598243,0.586667,124.000000
weighted avg,0.769501,0.741935,0.754194,124.000000
NR,0.848214,0.922330,0.883721,103.000000
R,0.333333,0.190476,0.242424,21.000000
accuracy,0.798387,0.798387,0.798387,0.798387
macro avg,0.590774,0.556403,0.563073,124.000000
weighted avg,0.761017,0.798387,0.775114,124.000000


## Voting Classifier

### Model training and Classification

In [30]:
estimator = []
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('SVC', svm.SVC(kernel='linear', gamma = 'auto', probability = True)))
estimator.append(('KNN', KNeighborsClassifier(n_neighbors=3)))
kf5 = StratifiedKFold(n_splits = 5, shuffle=False)
model3=AdaBoostClassifier(n_estimators=300, learning_rate=1.0)
i=1
dfs_vc = []
for train_index, test_index in kf5.split(X3,y):
    #select train and test datasets from X and y
    X_train, X_test = X3.iloc[train_index], X3.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #train the model
    vc = VotingClassifier(estimators = estimator, voting='hard').fit(X_train, y_train)
    #predict the test dataset
    predicted3 = vc.predict(X_test)
    #print the classification score report
    report = classification_report(y_test, predicted3, output_dict = True)
    df = pd.DataFrame(report).transpose()
    dfs_vc.append(df)
    i+=1
results_df_vc = pd.concat(dfs_vc)

In [31]:
results_df_vc

,precision,recall,f1-score,support
NR,0.851064,0.388350,0.533333,103.000000
R,0.181818,0.666667,0.285714,21.000000
accuracy,0.435484,0.435484,0.435484,0.435484
macro avg,0.516441,0.527508,0.409524,124.000000
weighted avg,0.737724,0.435484,0.491398,124.000000
NR,0.846154,0.854369,0.850242,103.000000
R,0.250000,0.238095,0.243902,21.000000
accuracy,0.750000,0.750000,0.750000,0.750000
macro avg,0.548077,0.546232,0.547072,124.000000
weighted avg,0.745192,0.750000,0.747555,124.000000
